In [8]:
import xml.etree.ElementTree as ET 

In [9]:
# Passing the path of the 
# xml document to enable the 
# parsing process 
tree = ET.parse('agh-fis-spr17.xml') 

In [10]:
# getting the parent tag of 
# the xml document 
root = tree.getroot() 

In [11]:
#Find and remove rooms tag
rooms = root.find('rooms')
root.remove(rooms)

In [12]:
#Find optimization tag
optimization = root.find('optimization')

In [13]:
#remove room atribute in optimization
optimization.attrib.pop('room')

'1'

In [14]:
#Find courses tags
courses = root.find('courses')

In [15]:
#Getting all classes from all courses
classes = courses.findall('./course/config/subpart/class')

In [16]:
#Removig all room children from each class parent
for Class in classes:
    if(Class.get('room') != None):
        Class.attrib.pop('room')
    rooms = Class.findall('./room')
    for room in rooms:
        Class.remove(room)

In [17]:
#Finding and removing same-room constraints
distributions = root.find('distributions')
samerooms = distributions.findall(".//distribution[@type='SameRoom']")

for dists in samerooms:
    distributions.remove(dists)

#Finding and removing different-room constraints    
diffrooms = distributions.findall(".//distribution[@type='DifferentRoom']")

for dists in diffrooms:
    distributions.remove(dists)

In [18]:
tree.write('agh-fis-spr17_trimmed.xml')

In [19]:
#Copying initial lines from input file
with open("agh-fis-spr17.xml") as myfile:
    head = [next(myfile) for x in range(1)]

with open('agh-fis-spr17_trimmed.xml', 'r') as trimmed:
    content = trimmed.read() 


with open('agh-fis-spr17_trimmed.xml', 'w') as trimmed:
    trimmed.seek(0,0)
    trimmed.write(head[0]+'\n'+content)